# Initialize

In [1]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
import datetime
import re
import csv
from IPython.display import display, HTML

In [2]:
# Read elements.csv file 
# Contains elemental info and K_a windows
elements = pd.read_csv('elements2.csv')
elements.drop(elements.index[0], inplace=True)
elements = elements.set_index('Name')

# User Interface <a class="tocSkip">

In [3]:
# Create list for elements boxes format: AtomicNumber - Name
sample_elements_list = []
for i in elements.index.values:
    sample_elements_list.append(f"{int(elements.loc[i]['Number'])} - {i}")

In [4]:
'''
Set up for GUI to enter in data.
Outputs (access values with name.value)
sample_set, material_system, density
sample_elements, probe_current, GB_calcs.value
emsa_folder, sigmaK_file, flash_time
'''

# Sample Description
sample_set = widgets.Text(
    value='B4C Grain Boundaries',
    placeholder='Type something',
    description='Sample Description:',
    style={'description_width': 'initial'},
    disabled=False
)
# Material System
material_system = widgets.Text(
    value='B4CMEDE',
    placeholder='Type something',
    description='Material System:',
    style={'description_width': 'initial'},
    disabled=False
)
# Density 
density = widgets.FloatText(
    value=2.52,
    description='Density (g/cm3):',
    style={'description_width': 'initial'},
    disabled=False
)
# Elements for Atomic Numbers
sample_elements = widgets.SelectMultiple(
    options=list(elements.sort_index().index),
    value=['Boron', 'Carbon', 'Oxygen', 'Silicon'],
    rows=8,
    description='Elements in Sample:',
    style={'description_width': 'initial'},
    disabled=False
)

#Flash time hour and minutes
flash_minutes = widgets.BoundedIntText(
    value=13,
    min=0,
    max=24,
    step=1,
    description='Mins (0-60):',
    disabled=False
)

flash_seconds = widgets.BoundedIntText(
    value=5,
    min=0,
    max=59,
    step=1,
    description='Sec (0-59):',
    disabled=False
)
# Probe Current
probe_current = widgets.FloatText(
    value=190,
    description='Probe Current (pA):',
    style={'description_width': 'initial'},
    disabled=False
)
# Checkbox for GB Excess Calcs
GB_calcs = widgets.Checkbox(
    value=False,
    description='Check to Include GB Excess Calculations',
    style={'description_width': 'initial'},
    disabled=False
)

# Folder where emsa files are
emsa_folder = widgets.Text(
    value='./emsa_files',
    placeholder='Type something',
    description='EMSA files location:',
    style={'description_width': 'initial'},
    disabled=False
)

# Location of 2sigmaK file
sigmaK_file = widgets.Text(
    value='2sigmaK.zfe',
    placeholder='Type something',
    description='Name of 2sigmaK file:',
    style={'description_width': 'initial'},
    disabled=False
)
# Mn Peak Energy
Mn_peak_energy = widgets.FloatText(
    value=5898,
    description='Mn Peak Energy (eV):',
    style={'description_width': 'initial'},
    disabled=True
)
# Mn FWHM
Mn_FWHM = widgets.FloatText(
    value=131.6,
    description='Mn FWHM:',
    style={'description_width': 'initial'},
    disabled=True
)

## Display Gui Below

In [5]:
'''
Creates User Interface to enter in data.
Outputs (access values with name.value)
sample_set, material_system, density
sample_elements, probe_current, 
emsa_folder, sigmaK_file, flash_minutes,
flash seconds, Mn_peak_energy, Mn_FWHM

GB_calcs (checkbox)
'''
leftbox = widgets.VBox([material_system, sample_set, density, probe_current])
rightbox = widgets.VBox([sample_elements])
# Combine left and right box
topbox = widgets.HBox([leftbox, rightbox])
# Add flashtime box with label
middlebox1 = widgets.HBox([widgets.Label('Flash time: '),flash_minutes, flash_seconds])
middlebox2 = widgets.HBox([Mn_peak_energy, Mn_FWHM])
bottombox = widgets.HBox([emsa_folder, sigmaK_file])
#stack top and bottom box
interface = widgets.VBox([topbox, middlebox1, middlebox2, bottombox, GB_calcs])
display(interface)

# Read file info

## Read ZFE   

In [6]:
"""
Reads .zfe file, skips comment headers preceded by # in readZFE(),
Parses and extracts data from each line in splitdata()
Returns dataframe of zfe file info

Input: 'sigmaK_file.value'
Output: 
Dataframe of 
'atomic number','line', 'energy', 'zeta', 'zeta error'
with energy in (eV) and zeta factor in (kg/m2 electron/photon)
"""
def ZFE(filename):
        
    def readZFE(filename):
            # OpenZFE file and read into list
        with open(filename, 'r') as f:
            # Skip initial comments that starts with #
            while True:
                line = f.readline()
                # break while statement if it is not a comment line
                # i.e. does not startwith #
                if line.startswith('# -z'):
                    break
            # Process the rest of the file
            filedata = f.read().splitlines()
        return(splitdata(filedata))

    def splitdata(data):
        temp = []
        # Define column headers
        header = ['atomic number','line', 'energy', 'zeta', 'zeta error']
        for line in data:
            temp.append([line.split()[1], line.split()[3], line.split()[4],
                                   line.split()[5], line.split()[7]])
        dfZFE = pd.DataFrame(temp, columns=header)
        # Set columns to correct datatypes
        dfZFE = dfZFE.astype({'atomic number':'int32', 'line':'str', 'energy':'float32',
                     'zeta':'float32', 'zeta error':'float32'})
        #Multiply energy by 1000
        dfZFE['energy'] = dfZFE['energy']*1000
        return(dfZFE)
    
    ZFEdata = readZFE(filename)
    return(ZFEdata)

## Read EMSA files
Reads EMSA spectrums into a dataframe  
Returns: Spectrum info and metadata

In [7]:
'''
Input: open EMSA file, EMSAfilename from getEMSA
Output: array of metadata from file
'''
def getMetadata(open_file, filename):
    cols = ['collection_date', 'acq_time', 'x_tilt', 'y_tilt','live_time']
    info = []
    while True:
        line = open_file.readline()
        if line.startswith(('#DATE', '#XTILTSTG','#YTILTSTGE', '#TIME','#LIVETIME')):
            # Remove newline return at end
            line = line.rstrip('\n')
            info.append(line.split(': ', 1)[1])
        if line.startswith('#SPECTRUM'):
            break
    #Fix missing zeros in time formatting
    info = [sub.replace(': ', ':0') for sub in info]
    #Convert to DataFrame
    info = pd.DataFrame([info], columns=cols, index={filename:1})
    # Convert acq to seconds
    info['acq_time'] = pd.to_timedelta(info['acq_time'][0], unit='h').total_seconds() / 60
    
    # Format as datetime
    #info[info]
    #date_time_obj = datetime.datetime.strptime(metadata[2][1], '%H:%M:%S')
    
    return(info)

In [8]:
'''
Input: open EMSA file, EMSAfilename from getEMSA
Output: dataframe of spectrum from file
'''
# Create dataframe with column names as energy bins and index as filename
def getSpectra(open_file, filename):
    # Read files data
    spectrum = pd.read_csv(open_file, header=None, usecols=[0,1])
   
    # Delete Rows with NA (eg: #END OF FILE)
    spectrum = spectrum[~spectrum[0].str.contains("#")]
    spectrum[0] = spectrum[0].astype(float) #Remove excess zeros in bins
    spectrum = spectrum.T
        
    # Make Column names=bin (eV), data=counts, index=filename
    spectrum.columns = spectrum.iloc[0]
    spectrum.drop(0, inplace=True)
    
    # Make index the file name
    spectrum.rename(index={1:filename}, inplace=True)
    return(spectrum)

In [9]:
'''
Input: EMSA_folder input from GUI
Output: list of EMSA filenames, dataframe metadata, dataframe of spectrum
'''
def getEMSA(emsa_folder):
    # Select on files that end in .emsa in the provided directory
    emsa_names = [f for f in os.listdir(emsa_folder.value) if f.endswith('.emsa')]
    metadata = pd.DataFrame()
    spectrum = pd.DataFrame()
    #Loop thru files - get metadata and spectra
    for filename in emsa_names:
        with open(os.path.join(emsa_folder.value + '/' + filename), 'r') as f:
            #Get Metadata from the file
            metadata= metadata.append(getMetadata(f, filename))
            
            # Get Spectra
            spectrum = spectrum.append(getSpectra(f, filename))
    return(emsa_names, metadata, spectrum)

## Run File Reading Scripts

In [10]:
# Read ZFE file
# columns: atomic number, line, energy, zeta, zeta error
zfe_data = ZFE(sigmaK_file.value)

# Read EMSA files
# emsa_filenames - list of filenames in order they're read in
# Indexed by filename:
# metadata - collection_date, acq_time,x_tilt,y_tilt,live_time
# spectrum - header is in eV
emsa_filenames, metadata, spectrum = getEMSA(emsa_folder)

## Check Live Time and Tilt

In [11]:
# Check that the live time and the microscope x-tilt and y-tilts are the same for all spectra
# These parameters must be constant to generate accurate results, so if
# they are not constant an error message is display

if len(metadata['live_time'].astype(float).round(1).unique()) !=1: 
    display(HTML("<script>window.alert('Spectrum Error: Not all the live times are the same');</script>"))
if len(metadata['x_tilt'].astype(float).unique()) !=1: 
    display(HTML("<script>window.alert('Spectrum Error: Not all the x-tilts are the same');</script>"))
if len(metadata['y_tilt'].astype(float).unique()) !=1: 
    display(HTML("<script>window.alert('Spectrum Error: Not all the y-tilts are the same');</script>"))

# Calculate Counts

## Get windows from 'elements'

In [12]:
# Fetches the window ranges from 'elements', originally from elements.csv
# Indexed by element
# Order of columns: window_start, window_end, background1, background2, background3...etc
num_cols = np.size(eval(elements.loc[sample_elements.value, material_system.value][0]))
windows = pd.DataFrame(columns=[list([i for i in range(num_cols)])])
for i in sample_elements.value:
    row = eval(elements.loc[i][material_system.value])
    windows.loc[i,:] = list(np.array(row).flat)
# Adjust starting window
windows = windows.round(2)

## Get Counts

In [13]:
counts = pd.DataFrame()
for i in sample_elements.value:
    w1 = np.round(windows.loc[i, 0].values[0], 2) # Start of peak
    w2 = np.round(windows.loc[i, 1].values[0], 2) # End of peak window
    counts.loc[:, f"{i}_peak"] = spectrum.loc[:, w1:w2].sum(axis=1)
    
    # Background counts
    b1start = np.round(windows.loc[i, 2].values[0], 2)
    b1end = np.round(windows.loc[i, 3].values[0], 2)
    b1 = spectrum.loc[:, b1start:b1end].sum(axis=1)
    
    b2start = np.round(windows.loc[i, 4].values[0], 2)
    b2end = np.round(windows.loc[i, 5].values[0], 2)
    b2 = spectrum.loc[:, b2start:b2end].sum(axis=1)
    
    counts.loc[:, f"{i}_background"] = (b1 + b2)/2
    
    # Raw intensity values
    stat_factor =  counts[f"{i}_peak"] - counts[f"{i}_background"]
    # Check if statistically significant background
    stat_factor.where(stat_factor > 3*np.sqrt(2*counts[f"{i}_background"]), 0)
    raw_peak_intensity = counts[f"{i}_peak"] - counts[f"{i}_background"]

    # Correct Peak Intensity
    current_decay_time = metadata['acq_time'] - (flash_minutes.value*60 + flash_seconds.value)
    current_adjustment = 189/(189 - 0.086*current_decay_time);
    peak_intensity = raw_peak_intensity * current_adjustment;
    counts.loc[:, f"{i}_peak_intensity"] = peak_intensity
    
        

# Print FORTRAN Input Files

## Print ukn file

In [14]:
# Print the counts to a ukn file for zeta factor code
# To run the zeta analysis code written by Masashi Watanabe, we had to
# create a .ukn (i.e. unknown) file that contains the peak intensities and energy windows

with open(f"{material_system.value}.ukn", 'w') as output:
    writer = csv.writer(output)
    
    # Print Header
    writer.writerow([f'# XEDS spectrum measurements from {material_system.value} {sample_set.value} using ARM200CF at Lehigh University'])
    writer.writerow(['# JEOL centurio detector'])
    writer.writerow([f"# Date: {metadata['collection_date'][0]}"])
    writer.writerow([f"# Acquisition time: {metadata['live_time'][0]}"])
    writer.writerow([f'# Probe Current: {probe_current.value} pA'])
    writer.writerow(["#"])

    # Print the Energy windows and Elements
    writer.writerow(['# Energy Window Information'])
    for i in sample_elements.value:
        ka1 = eval(elements.loc[i][material_system.value])[0][0]
        ka2 = eval(elements.loc[i][material_system.value])[0][1]
        writer.writerow([f"# -w {elements.loc[i]['Symbol']}_Ka {ka1}   {ka2}"])
    writer.writerow(['#'])

    # Print EMSA File Names
    writer.writerow(['# EDS File Names'])
    for i in emsa_filenames:
        writer.writerow([f"# {i}"])
    writer.writerow(['# '])

    # Print the header row of the counts table 
    # Based on length of sample_elements.value
    output.write("# \t ")
    for i in sample_elements.value:
        output.write(f"{elements.loc[i]['Symbol']}_Ka \t\t\t")
    output.write('\r\n')
    
    # Print the background subtracted intensities for each element
    # Set decimals to only two places
    for emsa in emsa_filenames:
        output.write("?     ")
        for i in sample_elements.value:
            value = counts.loc[emsa, f"{i}_peak_intensity"]
            output.write("{:0.2f} ".format(value))
        output.write('\r\n')
            

## Print BGI file

In [15]:
# Print the counts to a bgi file for zeta factor code
# To determine minimum mass fractions, this section outputs the background from each peak to input into the zeta analysis code written by Masashi Watanabe. 
# Here we create a .bgi (i.e. background intensity) file that contains the peak intensities and energy windows

with open(f"{material_system.value}.bgi", 'w') as output:
    writer = csv.writer(output)
    
    # Print Header
    writer.writerow([f'# XEDS spectrum measurements from {material_system.value} {sample_set.value} using ARM200CF at Lehigh University'])
    writer.writerow(['# JEOL centurio detector'])
    writer.writerow([f"# Date: {metadata['collection_date'][0]}"])
    writer.writerow([f"# Acquisition time: {metadata['live_time'][0]}"])
    writer.writerow([f'# Probe Current: {probe_current.value} pA'])
    writer.writerow(["#"])

    # Print the Energy windows and Elements
    writer.writerow(['# Energy Window Information'])
    for i in sample_elements.value:
        ka1 = eval(elements.loc[i][material_system.value])[0][0]
        ka2 = eval(elements.loc[i][material_system.value])[0][1]
        writer.writerow([f"# -w {elements.loc[i]['Symbol']}_Ka {ka1}   {ka2}"])
    writer.writerow(['#'])

    # Print EMSA File Names
    writer.writerow(['# EDS File Names'])
    for i in emsa_filenames:
        writer.writerow([f"# {i}"])
    writer.writerow(['# '])

    # Print the header row of the counts table 
    # Based on length of sample_elements.value
    output.write("# \t ")
    for i in sample_elements.value:
        output.write(f"{elements.loc[i]['Symbol']}_Ka \t\t\t")
    output.write('\r\n')
    
    # Print the background counts for each element
    for emsa in emsa_filenames:
        output.write("?     ")
        for i in sample_elements.value:
            value = counts.loc[emsa, f"{i}_background"]
            output.write("{:0.2f} ".format(value))
        output.write('\r\n')


## Print TXT file

In [16]:
metadata['collection_date'][0]

'22-JUL-2018'

In [17]:
"""
Create a text file to run the .zetact code written by Masashi Watanabe 

Instead of creating a .ukn, manually running the .zetact, and then
outputting the data, we create this file to run the .zetact within Matlab

All of the input is determined in the sections above and the order of the
input must be maintained
"""
with open("Run_Zeta.txt", 'w') as output:
    writer = csv.writer(output)
    
    writer.writerow('') # Hit enter key
    writer.writerow('1') # type in definition information
    writer.writerow([f"{material_system.value}"]) # Typer material system for specimen name
    writer.writerow([f"{len(sample_elements.value)}"]) # Number of elements in sample
    
    for i in elements.loc[list(sample_elements.value), 'Number']:
        writer.writerow([f'{int(i)}']) # Element atomic numbers
    writer.writerow('y') # Confirm that the elements are correct
    
    # if an element Z>19, the code will ask if the intended X-rays are
    # K alpha (which we always prefer to use)
    for i in elements.loc[list(sample_elements.value), 'Number']:
        if (i > 19): writer.writerow('1')
    
    writer.writerow('y') #  check is x-ray lines are correct
    writer.writerow('2') # determine speciman density using harmonic mean
    writer.writerow('1') # use Heinrich 1986 for mass absorbtion coefficients (MAC)
    writer.writerow('n') # do not check MAC
    writer.writerow('y') # DO save definition file
    writer.writerow('2') # in the current directory
    writer.writerow([f"{material_system.value}"]) # save the sample definition file (.dfu) as the material system
    
    writer.writerow('2') # we do NOT want to define detector parameters
    writer.writerow(['arm']) # this name matches the .det file in mwWorld in C:drive
    writer.writerow('n') # we do NOT want to change any parameters
    
    writer.writerow('1') # we want to manually input the experiment conditions
    writer.writerow([f"{metadata['collection_date'][0]}"]) # enter collection date
    writer.writerow(['200'])# enter microscope high tension (i.e. KV)
    writer.writerow([f"{metadata['x_tilt'][0]}"]) # enter the x-tilt (i.e. alpha tilt in degrees)
    writer.writerow([f"{metadata['y_tilt'][0]}"]) # enter the y-tile (i.e. beta tilt in degrees)
    writer.writerow('n') # we do NOT want to change any values
    writer.writerow('n') # we do NOT want to correct for beam drop (we already do so manually in GetCounts)
    
    writer.writerow([f"{probe_current.value/1000}"]) # enter probe current in nA
    writer.writerow([f"{metadata['live_time'][0]}"]) # enter live time in seconds
    writer.writerow(['131.6']) # enter detector energy resolution in eV
    writer.writerow(['10']) # enter eV/channel
    writer.writerow(['0']) # enter zero eV offset 
    writer.writerow(['2048']) # enter 2048 channels
    writer.writerow('n') # we do NOT want to change any values
    writer.writerow([f"{material_system.value}"]) # save the experimental conditions file (dfe) as the material system
    
    writer.writerow([f"{material_system.value}.ukn"]) # assign the .ukn file that we created above
    writer.writerow('y') # YES we would like to calculate the minimum mass fraction (MMF)
    writer.writerow('2') # choose 95% certainty to match errors of zeta factor
    writer.writerow('1') # directly type in zeta factors
    for i in elements.loc[list(sample_elements.value), 'Number']:
        writer.writerow(["{:0.3f}".format(zfe_data.loc[zfe_data['atomic number'] == i, 'zeta'].item())])
        writer.writerow(["{:0.3f}".format(zfe_data.loc[zfe_data['atomic number'] == i, 'zeta error'].item())])

    writer.writerow('y') # calculate the atoms and spatial resolution
    writer.writerow(['0.2']) # enter beam diameter in nm
    writer.writerow('2') # confirm full width half max (FWHM)
    writer.writerow('y') # create log file
    writer.writerow(['exit'])